In [6]:
# 데이터처리
from __future__ import print_function
import numpy as np
import pandas as pd
from IPython.display import display, HTML
# 머신러닝 알고리즘 및 평가
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import pylab
import scipy as sp
import sklearn.preprocessing as preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\pc-user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
############# 보스턴 주택가격 데이터 변수설명
#[01]  CRIM: 자치시(town) 별 1인당 범죄율  
#[02]  ZN: 25,000 평방피트를 초과하는 거주지역의 비율  
#[03]  INDUS: 비소매상업지역이 점유하고 있는 토지의 비율  
#[04]  CHAS: 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)  
#[05]  NOX: 10ppm 당 농축 일산화질소  
#[06]  RM: 주택 1가구당 평균 방의 개수  
#[07]  AGE: 1940년 이전에 건축된 소유주택의 비율  
#[08]  DIS: 5개의 보스턴 직업센터까지의 접근성 지수
#[09]  RAD: 방사형 도로까지의 접근성 지수  
#[10]  TAX: 10,000 달러 당 재산세율  
#[11]  PTRATIO: 자치시(town)별 학생/교사 비율  
#[12]  B: 1000(Bk-0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함.  
#[13]  LSTAT: 모집단의 하위계층의 비율(%)  [14]  MEDV 본인 소유의 주택가격(중앙값) (단위: $1,000)


In [11]:
############# 데이터 불러오기
dat_dir = 'C:/Users/pc-user/Desktop/yscho/Hanhwa/'
dat = pd.read_csv(dat_dir+'Regression_1_boston_housing.csv')
display(HTML(dat.to_html()))
dat = dat.drop('ID', axis = 1)
display(HTML(dat.to_html()))

In [1]:
############# 데이터 살펴보기
# Data format


In [2]:
############# 데이터 전처리
# split x variable to y variable


In [3]:
# 정규화


In [4]:
# scatter plot

In [5]:
# correlation analysis


In [7]:
############# 학습모델 구축을 위한 데이터 셋 나누기


In [8]:
############# 학습모델 구축


In [9]:
############# 모델 학습


In [10]:
############# 모델 평가
# Check the Model Assumption: residual by scatter plot


In [11]:
# Check the Model Assumption: residual by box plot


In [12]:
# Check the Model Assumption: normal distribution


In [13]:
############# 예측성능평가
# prediction y_train


In [14]:
# prediction y_test


In [32]:
# performance measure: MSE, MAE, MAPE, Rsquared

In [15]:
# MSE: Mean of Squared error


In [16]:
# MAE: Mean of Absolute error


In [35]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [17]:
# MAPE: Mean of absolute percent error


In [18]:
# R2 score


In [19]:
# residual


In [21]:
# Residual plot: Training and test data


In [42]:
################## Multiple Linear Regression with Logarithm
y_train_log = pd.DataFrame(np.log(y_train))
#y_test_log = pd.DataFrame(np.log(y_test))

In [22]:
# prediction with logarithm


In [23]:
# MSE: Mean of Squared error


In [47]:
############### Feature selection: backward elimination
# ref: https://github.com/chekoduadarsh/Backward-Elimination-in-Python
def backwardElimination(x, y):
    SL = 0.05
    elimed_Variable = []
    numVars = len(np.array(x)[0])
    for i in range(0, numVars):
        regressor_OLS = smf.OLS(y, np.array(x)).fit()
        maxVar = max(regressor_OLS.pvalues)
        if maxVar > SL:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j] == maxVar):
                    x = np.delete(np.array(x), j, 1)
                                        
        else:
            elimed_Variable.append(i)
            pass
    regressor_OLS.summary()
    return elimed_Variable

In [51]:
##### Feature selection: Stepwise selection
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included